In [19]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [21]:
query = "SELECT * FROM core_sale;"
df_salesmans = pd.read_sql(query, conn)

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
df_sales = df_salesmans.copy()


In [23]:
df_sales_copy = df_sales.copy()

In [24]:
df_sales_copy["year"] = df_sales_copy.apply(lambda row : row["date"].year, axis=1)
df_sales_copy["month"] = df_sales_copy.apply(lambda row : row["date"].month, axis=1)
df_sales_copy["day"] = df_sales_copy.apply(lambda row : row["date"].day, axis=1)
df_sales_copy.drop(columns=["date"], inplace=True)
df_sales_copy.drop(columns=["description"], inplace=True)
df_sales_copy.drop(columns=["status"], inplace=True)
df_sales_machine_learning = df_sales_copy.copy()

In [25]:
df_sales_copy.set_index(['year', 'month', 'day'], inplace=True)

In [26]:
df_groupby_sales = df_sales_copy.groupby(["year", "month"])
sales_data = {
    'income': df_groupby_sales['income'].sum(),
    'count': df_groupby_sales['id'].count()
}

df_sales_per_months = pd.DataFrame(sales_data)
df_sales_per_months

income  count
year month                   
2018 11          13.83    144
     12          23.70    138
2019 1          223.55    149
     2           83.30    106
     3          135.51     71
     4          547.46    201
     5          618.98    272
     6          818.84    222
     7          766.23    230
     8          783.65    236
     9          964.01    273
     10        1787.53    276
     11        4171.90    566
     12        5332.49    456
2020 1         9728.35    498
     2         7232.84    421
     3        20139.21    506
     4        11431.84    383
     5        23537.96    379
     6        18548.85    575
     7        30161.80    649
     8        31853.57    634
     9        56912.63    742
     10       48486.92    702
     11      110440.01    666
     12      158934.42    572
2021 1       150329.29    595
     2       304759.41    872
     3       279697.81    777
     4       393478.44    710
     5       372174.41    739
     6       349250.89    745
     7       429155.46    876
     8       428391.10    816
     9       594890.60    908
     10     1014162.42   1114
     11     1190850.35   1106
     12     1035430.80    838
2022 1      1130625.74   1011
     2       759701.15    491

In [27]:
df_groupby_sales_year = df_sales_copy.groupby(["year"])
sales_data = {
    'income': df_groupby_sales_year['income'].sum(),
    'count': df_groupby_sales_year['id'].count()
}

df_sales_per_year = pd.DataFrame(sales_data)

In [28]:
df_groupby_sales_day = df_sales_copy.groupby(["year", "month","day"])
sales_data = {
    'income': df_groupby_sales_day['income'].sum(),
    'count': df_groupby_sales_day['id'].count()
}

df_sales_per_day = pd.DataFrame(sales_data)

In [29]:
df_sales_per_day

income  count
year month day                 
2018 11    12       1.85     12
           13       1.41     24
           14       0.18      7
           15       0.07      2
           16       0.87     11
...                  ...    ...
2022 2     7    54038.81     25
           8    84926.41     71
           9    55083.06     50
           10   50886.90     50
           11   24623.41     37

[814 rows x 2 columns]

In [30]:

df_sales_per_months

income  count
year month                   
2018 11          13.83    144
     12          23.70    138
2019 1          223.55    149
     2           83.30    106
     3          135.51     71
     4          547.46    201
     5          618.98    272
     6          818.84    222
     7          766.23    230
     8          783.65    236
     9          964.01    273
     10        1787.53    276
     11        4171.90    566
     12        5332.49    456
2020 1         9728.35    498
     2         7232.84    421
     3        20139.21    506
     4        11431.84    383
     5        23537.96    379
     6        18548.85    575
     7        30161.80    649
     8        31853.57    634
     9        56912.63    742
     10       48486.92    702
     11      110440.01    666
     12      158934.42    572
2021 1       150329.29    595
     2       304759.41    872
     3       279697.81    777
     4       393478.44    710
     5       372174.41    739
     6       349250.89    745
     7       429155.46    876
     8       428391.10    816
     9       594890.60    908
     10     1014162.42   1114
     11     1190850.35   1106
     12     1035430.80    838
2022 1      1130625.74   1011
     2       759701.15    491

In [32]:

df_sales_per_year

,income,count
year,,
2018,37.53,282
2019,16233.45,3058
2020,527408.40,6727
2021,6542570.98,10096
2022,1890326.89,1502
